# Test Cavity Model



In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
import torch
import numpy as np
import gc

import h5py
import pandas as pd
from matplotlib import pyplot as plt

Import local packages

In [3]:
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), "../.."))

if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

from src.utils.logger import Logging
from src.utils.utils import lp_error
from src.utils.logger import Logging

from src.utils.plot_loss import plot_loss_history

Set various constant variables: model path, name, etc.

In [4]:
TEST_DATA_PKL = "../../data/cavity.mat"
TEST_CHECKPOINT_PATH = os.path.join(PROJECT_ROOT, "result/cavity")


TANH_NORM_DIST = "../../model/a2/cavity_tanh_normal_a2.pth"
TANH_PARAM_DIST = "../../model/a1/cavity_tanh_trainable_a1.pth"
BSPLINE_DIST = "../../model/a2/cavity_bspline_a2.pth"
GRBF_DIST = "../../model/a1/cavity_grbf_a1.pth"
FOURIER_DIST = "../../model/a1/cavity_fourier_a1.pth"
CHEBTSHEV_DIST = "../../model/a2/cavity_chebyshev_a2.pth"
JACOBI_DIST = "../../model/a1/cavity_jacobi_a1.pth"


MODEL_PATH_LIST = {
    "tanh": TANH_NORM_DIST,
    "param_tanh": TANH_PARAM_DIST,
    "grbf": GRBF_DIST,
    "bspline": BSPLINE_DIST,
    "chebyshev": CHEBTSHEV_DIST,
    "jacobi": JACOBI_DIST,
    "fourier": FOURIER_DIST,
}


SOLVER_TO_MODULE = {
    "grbf": "src.nn.grbf",
    "jacobi": "src.nn.jacobi",
    "KAN_BSpline": "src.nn.bspline",
    "KAN_Chebyshev": "src.nn.chebyshev",
    "param_tanh": "src.nn.tanh_parameterized",
    "MLP2": "src.nn.tanh",
    "fourier": "src.nn.fourier",
}

Create logger

In [5]:
logger = Logging(TEST_CHECKPOINT_PATH)
model_dirname = logger.get_output_dir()

## Load the Model and Test

In [6]:
data = h5py.File(TEST_DATA_PKL, "r")  # load dataset from matlab

domain = pd.DataFrame(data["cavity_internal"]).T.to_numpy()

#  t = tf.reshape(tstep,N_data)[:,0].T
time_ = domain[:, 0:1]
xfa = domain[:, 1:2]
yfa = domain[:, 2:3]
ufa = domain[:, 3:4]
vfa = domain[:, 4:5]
pfa = domain[:, 5:6]

In [7]:
for activation, model_path in MODEL_PATH_LIST.items():
    logger.print(f"MODEL_PATH {model_path}")
    # Load the state from the saved model
    state = torch.load(
        model_path,
    )
    config = state.get("config", {})
    solver = config.get("solver")

    # Extract model configuration from state
    model_activation_name = config.get("activation")
    model_architecture = config.get("network")
    loss_history = state.get("loss_history")

    # Dynamically import the correct module and class
    if solver in SOLVER_TO_MODULE:
        module = __import__(SOLVER_TO_MODULE[solver], fromlist=["PINNKAN"])
        PINNKAN = getattr(module, "PINNKAN")

        # Initialize fluid and solid models
        model = PINNKAN(model_architecture, model_activation_name).to("cpu")

    model.load_state_dict(state["model_state_dict"])
    logger.print(f"model loaded from {model_path}")
    logger.print(f"problem: {config.get('problem')}")
    logger.print(f"dataset_path:  , {config.get('dataset_path')}")
    logger.print(f"batch size:  , {config.get('batch_size')}")
    logger.print(f"network:  , {config.get('network')}")
    logger.print(f"term loss weights :  , {config.get('weights')}")
    logger.print(f"activation:  , {config.get('activation')}")
    logger.print(f"solver:  , {config.get('solver')}")
    logger.print(
        f"number of iterations:  , {len(loss_history[next(iter(loss_history))])}"
    )

    test_torch_data = torch.tensor(
        np.concatenate([time_, xfa, yfa], axis=1), dtype=torch.float32
    ).to("cpu")
    with torch.no_grad():
        predictions = model.forward(test_torch_data)
    if predictions.is_cuda:
        predictions = predictions.cpu()
    u_pred = predictions[:, 0:1].numpy()
    v_pred = predictions[:, 1:2].numpy()
    p_pred = predictions[:, 2:3].numpy()

    text = "RelL2_"
    # logger.print("\n Relative L2 ERROR:")
    u_error2 = lp_error(u_pred, ufa, (text + "U%"), logger, 2)
    v_error2 = lp_error(v_pred, vfa, (text + "V%"), logger, 2)
    p_error2 = lp_error(p_pred, pfa, (text + "P%"), logger, 2)

    for key in loss_history:
        logger.print("Final loss %s: %e" % (key, loss_history[key][-1]))

    logger.print("******************************\n")

    # Delete model and clear cache after each iteration
    del model
    torch.cuda.empty_cache()
    gc.collect()  # Force garbage collection to release memory

logger.print("file directory:", logger.get_output_dir())

INFO:src.utils.logger:MODEL_PATH ../../model/a2/cavity_tanh_normal_a2.pth


INFO:src.utils.logger:model loaded from ../../model/a2/cavity_tanh_normal_a2.pth
INFO:src.utils.logger:problem: None
INFO:src.utils.logger:dataset_path:  , None
INFO:src.utils.logger:batch size:  , 128
INFO:src.utils.logger:network:  , [3, 300, 300, 300, 3]
INFO:src.utils.logger:term loss weights :  , [2.0, 2.0, 2.0, 2.0, 2.0, 0.1]
INFO:src.utils.logger:activation:  , tanh2
INFO:src.utils.logger:solver:  , MLP2
INFO:src.utils.logger:number of iterations:  , 60001
INFO:src.utils.logger:RelL2_U%  : 1.080e+01 
INFO:src.utils.logger:RelL2_V%  : 2.206e+01 
INFO:src.utils.logger:RelL2_P%  : 2.445e+01 
INFO:src.utils.logger:Final loss lleft: 2.178410e-03
INFO:src.utils.logger:Final loss lright: 4.574872e-04
INFO:src.utils.logger:Final loss lbottom: 1.525999e-04
INFO:src.utils.logger:Final loss lup: 2.957572e-03
INFO:src.utils.logger:Final loss linitial: 6.859205e-03
INFO:src.utils.logger:Final loss lphy: 2.197395e-02
INFO:src.utils.logger:******************************

INFO:src.utils.logger:

In [ ]:
    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Total number of trainable parameters in the model: {total_params}")

In [ ]:
pythom -m src.trainer.main_trainer --total_epochs 100 --save_every 10 --print_every 10 --batch_size 32 --log_path ./logs --solver param_tanh --problem cavity --weights [10, 10, 10, 10] --network [2, 10, 10, 1] --dataset_path ./data/cavity.h5